In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from __future__ import division

In [3]:
# form mzp6.ipnyb
class SystemSolver():
    def __init__(self,updates = None):
        if isinstance(updates,list):
            max_len = max([len(x) for x in updates])
            [x.extend([0]*(max_len-len(x))) for x in updates]
            self.updates = np.asarray(updates, dtype = np.float64)
        else:
            self.updates = updates
        
    @property
    def update_count(self):
        return min(self.updates.shape)-1
    
    def __call__(self,tn,yn,functions,h):
        k = np.zeros(shape=(self.update_count,len(functions)))
        for i in range(self.update_count):
            a = self.updates[i,0]
            b = self.updates[i,1:]
            upd = np.sum(b[:,np.newaxis]*k,axis = 0)
            for j in range(len(functions)):
                k[i,j]= h * functions[j](tn+a*h, yn+upd)
        c = self.updates[self.update_count,1:]
        if self.updates.shape[0]>self.updates.shape[1]:
            c_= self.updates[self.update_count+1,1:]
            return np.sum(c[:,np.newaxis]*k, axis = 0), np.sum(c_[:,np.newaxis]*k, axis = 0)
        return np.sum(c[:,np.newaxis]*k, axis = 0)

In [4]:
def get_spring(start,end,count,size = 1.):
    start = np.asarray(start)
    end = np.asarray(end)
    length = np.sqrt(np.sum((end-start)**2))
    flat_space = length/6.
    x = np.linspace(flat_space,length-flat_space,count*4+1)
    length_needed = 2*np.pi*count 
    s = size*np.sin((length_needed/(length-flat_space*2))*x)
    rotations = (end-start)/length
    rotation_matrix = np.array([[rotations[0],-rotations[1]],[rotations[1],rotations[0]]])
    first_flat = np.array([[0],[0]])
    second_flat = np.array([[length],[0]])
    return rotation_matrix.dot(np.hstack((first_flat, np.array([x,s]), second_flat))) + start[:,np.newaxis]

In [5]:
fehlberg_4_5 = SystemSolver([[0],
                            [1/4,1/4],
                            [3/8,3/32,9/32],
                            [12/13,1932/2197,-7200/2197,7296/2197],
                            [1,439/216,-8,3680/513,-845/4104],
                            [1/2,-8/27,2,-3544/2565,1859/4104,-11/40],
                            [0,25/216,0,1408/2565,2197/4104,-1/5,0],
                            [0,16/135,0,6656/12825,28561/56430,-9/50,2/55]])
fehlberg_4_5.ranks = (4,5)

In [6]:
def spring_pendulum(method, y_0s, approx, step = 0.1, filename = None):
    import matplotlib.animation as animation
    
    fig = plt.figure(figsize=(5,5))
    ax = fig.add_subplot(111)
    ax.grid()
    plt.ylim(-20,20)
    plt.xlim(-20,20)

    line, = ax.plot([], [], '-', lw=2)
    line2, = ax.plot([0],[0], '--', lw=1)
    
    time_template = 'time = %.1fs'
    time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
    
    mass, = ax.plot([],[], marker='o', markersize = 10.)
    
    old_x = []
    old_y = []
    def init():
        global Y
        Y = y_0s
        line.set_data([],[])
        line2.set_data([],[])
        time_text.set_text('')
        
        mass.set_data([],[])
        
        return line, line2, time_text, mass
    def animate(t):
        global Y,L
        y_1, y_2 = method(tn=t*step,yn=Y,functions=approx,h=step)
        Y += y_2
        new_x = (L+Y[0])*np.sin(Y[2])
        new_y = -(L+Y[0])*np.cos(Y[2])
        spring = get_spring([0.,0.],[new_x,new_y],6)
        line.set_data(spring[0],spring[1])
        old_x.append(new_x)
        old_y.append(new_y)
        line2.set_data(old_x,old_y)
        mass.set_data([new_x],[new_y])

        time_text.set_text(time_template % (t*step))
        return line, line2, time_text, mass
    
    ani = animation.FuncAnimation(fig, animate, np.arange(400),
                              interval=25, blit=True, init_func=init)

    if filename:
        ani.save(filename + '.mp4', fps=30)
    plt.show()

Ustawienie wartości globalnych

In [7]:
Y = np.zeros(shape=(4,))

Zagadnienie poczatkowe

In [8]:
x_0 = 0.
v_0 = 0.
theta_0 = 2.
omega_0 = 0.

Dane wahadła

In [9]:
L = 7.
m = 2.
g = 9.81
k = 20.

Rownania

In [10]:
functions = [lambda t,ys: ys[1],
          lambda t,ys: (L+ys[0])*(ys[3])**2+m*g*np.cos(ys[2])-k*ys[0]/m,
          lambda t,ys: ys[3],
          lambda t,ys: (-2*ys[1]*ys[3]-g*np.sin(ys[2]))/(L+ys[0])]

In [11]:
# form mzp6.ipnyb
def draw_system(method, t_0, y_0s, approx, exact = None, labels=None, step = 0.1):
    count = np.ceil(10.0/step)
    X = np.linspace(t_0,t_0+10,count)
    Y = np.zeros(shape=(len(approx),X.shape[0]))
    if exact:
        RY = np.asarray([y(X) if y is not None else None for y in exact])
    Y[:,0] = np.asarray(y_0s)
    
    for i in np.arange(1,count):
        Y_updates = method(tn=X[i-1],yn=Y[:,i-1],functions=approx,h=step)
        if isinstance(Y_updates,tuple):
            Y[:,i] = Y[:,i-1] + Y_updates[1]
        else:
            Y[:,i] = Y[:,i-1] + Y_updates
    
    figure_count = len(approx)
    if exact:
        figure_count += len(filter(lambda x: x is not None,exact))
    figure = 1
    plt.figure(figsize = (9,figure_count*3))
    for i in range(len(approx)):
        ax = plt.subplot(figure_count,1,figure)
        if labels and labels[i]:
            ax.set_title(labels[i])
        plt.plot(X,Y[i], color = 'blue', label ='Approx')
        if exact and exact[i]:
            plt.plot(X,RY[i], color = 'red', linestyle='--', label='Exact')
        plt.legend(loc = 'upper left')
        figure += 1

        if exact and exact[i]:
            plt.subplot(figure_count,1,figure)
            plt.plot(X,np.abs(RY[i]-Y[i]), color = 'blue', label= 'Global error')
            plt.legend(loc = 'upper left')
            figure+=1
            
    plt.show()

In [12]:
draw_system(fehlberg_4_5, 0., y_0s=[x_0,v_0,theta_0,omega_0], approx = functions, labels = ["x(t)","v(t)","Theta(t)","Omega(t)"])

In [13]:
spring_pendulum(method=fehlberg_4_5,
               y_0s=[x_0,v_0,theta_0,omega_0],
               approx= functions,
               step=0.05,
               filename = 'spring_3')

In [14]:
x_0 = 0.
v_0 = -30.
theta_0 = .2
omega_0 = 5.

In [15]:
spring_pendulum(method=fehlberg_4_5,
               y_0s=[x_0,v_0,theta_0,omega_0],
               approx= functions,
               step=0.05,
               filename = 'spring_5')

In [16]:
x_0 = -5.
v_0 = 0.
theta_0 = 3.12
omega_0 = 0.

In [17]:
spring_pendulum(method=fehlberg_4_5,
               y_0s=[x_0,v_0,theta_0,omega_0],
               approx= functions,
               step=0.05,
               filename = 'spring_6')

Wahadło podwójne

In [18]:
Y = np.zeros(shape=(4,))

Zagadnienie poczatkowe

In [19]:
theta_1_0 = 0.
omega_1_0 = 0.
theta_2_0 = 2.
omega_2_0 = 0.

Dane wahadła

In [20]:
L_1 = 10.
L_2 = 5.
m_1 = 2.
m_2 = 1.

Równania

In [21]:
functions = [
    lambda t,ys: ys[1],
    lambda t,ys: (-m_2*L_1*ys[1]**2*np.sin(ys[0]-ys[2])*np.cos(ys[0]-ys[2]) + \
                  g*m_2*np.sin(ys[2])*np.cos(ys[0]-ys[2]) - \
                  m_2*L_2*ys[3]**2*np.sin(ys[0]-ys[2]) - \
                  (m_1+m_2)*g*np.sin(ys[0])) / \
    (L_1*(m_1+m_2) - m_2*L_1*np.cos(ys[0]-ys[2])**2),
    lambda t,ys: ys[3],
    lambda t,ys: (m_2*L_2*ys[3]**2*np.sin(ys[0]-ys[2])*np.cos(ys[0]-ys[2]) + \
                 g*np.sin(ys[0])*np.cos(ys[0]-ys[2])*(m_1+m_2) + \
                  L_1*ys[1]**2*np.sin(ys[0]-ys[2])*(m_1+m_2) - \
                  g*np.sin(ys[2])*(m_1+m_2)) / \
    (L_2*(m_1+m_2) - m_2*L_2*np.cos(ys[0]-ys[2])**2)
]

In [22]:
draw_system(fehlberg_4_5,
            0.,
            y_0s=[theta_1_0,omega_1_0,theta_2_0,omega_2_0],
            approx = functions,
            labels = ["Theta_1(t)","Omega_1(t)","Theta_2(t)","Omega_2(t)"])

In [23]:
def double_pendulum(method, y_0s, approx, step = 0.1, saving = False):
    import matplotlib.animation as animation
    
    fig = plt.figure(figsize=(5,5))
    ax = fig.add_subplot(111)
    ax.grid()
    plt.ylim(-20,20)
    plt.xlim(-20,20)

    line, = ax.plot([], [], 'o-', lw=2)
    line2, = ax.plot([0],[0], '--', lw=1)
    time_template = 'time = %.1fs'
    time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
    old_x = []
    old_y = []
    def init():
        global Y
        Y = y_0s
        line.set_data([],[])
        line2.set_data([],[])
        time_text.set_text('')
        
        return line, line2, time_text
    
    def animate(t):
        global Y,L_1,L_2
        y_1, y_2 = method(tn=t*step,yn=Y,functions=approx,h=step)
        Y += y_2
        new_x_1 = (L_1)*np.sin(Y[0])
        new_y_1 = -(L_1)*np.cos(Y[0])
        new_x_2 = new_x_1 + (L_2)*np.sin(Y[2])
        new_y_2 = new_y_1 -(L_2)*np.cos(Y[2])
        line.set_data([0.0,new_x_1,new_x_2],[0.0,new_y_1,new_y_2])
        old_x.append(new_x_2)
        old_y.append(new_y_2)
        line2.set_data(old_x,old_y)

        time_text.set_text(time_template % (t*step))
        return line, line2, time_text
    
    ani = animation.FuncAnimation(fig, animate, np.arange(400),
                              interval=25, blit=True, init_func=init)

    if saving:
        ani.save('pendulum.mp4', fps=30)
    plt.show()

In [24]:
double_pendulum(method=fehlberg_4_5,
               y_0s=[theta_1_0,omega_1_0,theta_2_0,omega_2_0],
               approx= functions,
               step=0.05)

In [25]:
theta_1_0 = 3.
omega_1_0 = 0.
theta_2_0 = 0.
omega_2_0 = 0.

In [26]:
double_pendulum(method=fehlberg_4_5,
               y_0s=[theta_1_0,omega_1_0,theta_2_0,omega_2_0],
               approx= functions,
               step=0.05)

In [27]:
theta_1_0 = 3.
omega_1_0 = 0.
theta_2_0 = 0.
omega_2_0 = 10.

In [28]:
double_pendulum(method=fehlberg_4_5,
               y_0s=[theta_1_0,omega_1_0,theta_2_0,omega_2_0],
               approx= functions,
               step=0.05)

In [29]:
def merson(tn,x,function,h):
    k1 = h*function(tn,x)
    k2 = h*function(tn+h/3,x+k1/3)
    k3 = h*function(tn+h/3,x+(k1+k2)/6)
    k4 = h*function(tn+h/2,x+(k1+k3*3)/8)
    k5 = h*function(tn+h,x+k1/2-3*k3/2+2*k4)
    return k1/6+2*k4/3+k5/6

In [30]:
spring_len = lambda direction,ballz: np.sqrt(np.sum((direction-ballz)**2,axis = 0))
def update(tn,x):
    balls = x[0,:,1:]
    north = x[0,:,:-1]
    
    spring_n = spring_len(north,balls)
    
    update_north = (spring_n-L)*(balls-north)/spring_n
    
    result = np.zeros_like(x)
    result[1,:,1:] = -(k/m)*(update_north) - np.sign(x[1,:,1:])*c*x[1,:,1:]**2 -c_*x[1,:,1:]
    result[1,1,1:] -= g
    result[0] = x[1]
    
    return result

In [50]:
def multiple_pendulum(method, step = 0.01):
    import matplotlib.animation as animation
    
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
    ax.grid()
    plt.ylim(-100,100)
    plt.xlim(-100,100)

    time_template = 'time = %.1fs'
    time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
    
    pendulum, = ax.plot([], [], '-o', lw=2)
    
    pos_y = np.arange(30)*.1
    pos_x = -np.arange(30)*.1
    pos = np.array([pos_x,pos_y])
    vel = np.zeros_like(pos)
    vel[:,1:] += 1.
    
    global state, frame
    state = np.array([pos,vel])
    frame = 0.
    def init():
        global state
        
        pendulum.set_data([],[]) 
        
        return pendulum, time_text
    
    def animate(t):
        global state,frame
        while frame < 1/30:
            state += merson(tn=t*step,x=state,function=update,h=step)
            frame += step
        frame -= 1/30
      
        
        pendulum.set_data(state[0,0],state[0,1])
        time_text.set_text(time_template % (t*step))
        return pendulum, time_text
    
    ani = animation.FuncAnimation(fig, animate, np.arange(np.floor(10/step)),
                              interval=33.3, blit=True, init_func=init)

    ani.save('pendulum-5.mp4', fps=30)
    plt.show()
    return ani

In [51]:
L = 0.1
g = 9.81
k = 30
m = 10
c = 0.05
c_ = 0.01

In [52]:
ani = multiple_pendulum(method=merson,step=0.01)